In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/haryoaw/documents/courses/nlp802/project/texteditalay


In [2]:
import fire
from transformers import AutoTokenizer, BertForTokenClassification, BertConfig, BertForMaskedLM
from neo_stif.components.utils import create_label_map
import pandas as pd
from neo_stif.components.train_data_preparation import prepare_data_tagging_and_pointer
import datasets
from neo_stif.lit import LitPointer, LitTaggerOrInsertion
from torch.utils.data import DataLoader
from neo_stif.components.collator import FelixCollator, FelixInsertionCollator
from lightning import Trainer
from lightning.pytorch.callbacks import RichProgressBar, ModelCheckpoint, EarlyStopping
from neo_stif.components.utils import compute_class_weights
from datasets import load_from_disk


MAX_MASK = 30
USE_POINTING = True


model_dict = {"koto": "indolem/indobert-base-uncased"}


LR_TAGGER = 5e-5 # due to the pre-trained nature
LR_POINTER = 1e-5 # no pre-trained
LR_INSERTION = 2e-5 # due to the pre-trained nature
VAL_CHECK_INTERVAL = 20

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
label_dict = create_label_map(MAX_MASK, USE_POINTING)

# Callback for trainer

df_train = pd.read_csv("data/stif_indo/train_with_pointing.csv")
data_train = datasets.Dataset.from_pandas(df_train)
data_train, label_dict = prepare_data_tagging_and_pointer(
    data_train, tokenizer, label_dict
)
model_path_or_name = model_dict["koto"]

Map: 100%|██████████| 1922/1922 [00:01<00:00, 1319.10 examples/s]


In [7]:
processed_train_data = "data/stif_indo/train_insertion"
processed_dev_data = "data/stif_indo/dev_insertion"
batch_size=2
device="cpu"
rich_cb = RichProgressBar()

ea_stop = EarlyStopping(patience=5, monitor="val_loss", mode="min")
train_data = load_from_disk(processed_train_data)
dev_data = load_from_disk(processed_dev_data)
train_dl = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=FelixInsertionCollator(tokenizer),
)
dev_dl = DataLoader(
    dev_data,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=FelixInsertionCollator(tokenizer),
)
model = BertForMaskedLM.from_pretrained(model_path_or_name)
lit_insert = LitTaggerOrInsertion(
    model,
    lr=LR_INSERTION,
    num_classes=model.config.vocab_size,
    class_weight=None,
    tokenizer=tokenizer,
    label_dict=label_dict,
    is_insertion=True,
)
trainer = Trainer(
    accelerator=device,
    devices=1,
    val_check_interval=20,
    check_val_every_n_epoch=None,
    callbacks=[rich_cb, ea_stop],
    fast_dev_run=True
)


Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accele

In [8]:
trainer.fit(lit_insert, train_dl, dev_dl)

┏━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name    ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model   │ BertForMaskedLM  │  110 M │
│ 1 │ ce_loss │ CrossEntropyLoss │      0 │
└───┴─────────┴──────────────────┴────────┘

Trainable params: 110 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 110 M                                                                                                
Total estimated model params size (MB): 442

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:490: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers`

31923

Input before going to output: [('[CLS]', 'IGNORED'), ('[MASK]', 'admin'), ('[UNK]', 'IGNORED'), ('min', 'IGNORED'),
('[PAD]', 'IGNORED'), (',', 'IGNORED'), ('promo', 'IGNORED'), ('"', 'IGNORED'), ('makan', 'IGNORED'), (',', 
'IGNORED'), ('jajan', 'IGNORED'), ('bareng', 'IGNORED'), ('"', 'IGNORED'), ('[MASK]', 'jika'), ('[MASK]', 'belum'),
('[MASK]', 'premium'), ('[UNK]', 'IGNORED'), ('kalo', 'IGNORED'), ('bel', 'IGNORED'), ('##om', 'IGNORED'), 
('premi', 'IGNORED'), ('##un', 'IGNORED'), ('[PAD]', 'IGNORED'), (',', 'IGNORED'), ('cash', 'IGNORED'), ('##back', 
'IGNORED'), ('[MASK]', '##nya'), ('[MASK]', 'dapat'), ('[UNK]', 'IGNORED'), ('nya', 'IGNORED'), ('dapet', 
'IGNORED'), ('[PAD]', 'IGNORED'), ('berapa', 'IGNORED'), ('?', 'IGNORED'), ('[SEP]', 'IGNORED')]

Input, pred: [('[CLS]', ','), ('[MASK]', ','), ('[UNK]', '[UNK]'), ('min', 'min'), ('[PAD]', ')'), (',', ','), 
('promo', 'promo'), ('"', '"'), ('makan', 'makan'), (',', ','), ('jajan', 'jajan'), ('bareng', 'bareng'), ('"', 
'"'), ('[MASK]', ','), ('[MASK]', '.'), ('[MASK]', '##back'), ('[UNK]', '[UNK]'), ('kalo', 'kalo'), ('bel', 'bel'),
('##om', '##om'), ('premi', 'premi'), ('##un', '##un'), ('[PAD]', ','), (',', ','), ('cash', 'cash'), ('##back', 
'##back'), ('[MASK]', '##nya'), ('[MASK]', ','), ('[UNK]', '[UNK]'), ('nya', 'nya'), ('dapet', 'dapet'), ('[PAD]', 
'berapa'), ('berapa', 'berapa'), ('?', '?'), ('[SEP]', '.')]

`Trainer.fit` stopped: `max_steps=1` reached.
